## Import

In [ ]:
import cv2 as cv
import numpy as np
from collections import Counter
import copy

import base64
import dash
import dash_daq as daq
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from dash import html as html
from dash import dcc
import plotly.graph_objs as go
import plotly.express as px
from PIL import Image
from io import BytesIO as _BytesIO
import os

## YOLOv3 Inference

#### Download YOLOv3 Weights and Configuration files from the below links:    

https://pjreddie.com/media/files/yolov3.weights

https://opencv-tutorial.readthedocs.io/en/latest/_downloads/10e685aad953495a95c17bfecd1649e5/yolov3.cfg

In [ ]:
def YOLO(img):
    
    # Load the YOLO network model
    net = cv.dnn.readNetFromDarknet('yolov3.cfg', 'yolov3.weights')
    net.setPreferableBackend(cv.dnn.DNN_BACKEND_OPENCV)

    # preprocess the input image
    scaled_img = cv.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
    
    # Set the input for the YOLO network
    net.setInput(scaled_img)
    
    # Run the Forward pass and select the output layers
    yolo_outputs = net.forward(['yolo_82', 'yolo_94', 'yolo_106'])
    
    BBoxs = []
    confidences = []
    classIDs = []
    img_H = img.shape[0]
    img_W = img.shape[1]
    
    for layer_output in yolo_outputs:
        for detected_box in layer_output:
            class_scores = detected_box[5:]
            classID = np.argmax(class_scores)
            confidence = class_scores[classID]
            if confidence > 0.5:
                
                ## Get the BBox parameters relative to img W & H
                box_relative = detected_box[:4]
                
                ## Scale the BBox parameters to get the absolute values
                BBox= box_relative * np.array([img_W, img_H, img_W, img_H])
                (Xc, Yc, W, H) = BBox.astype("int")
                
                ## Get the BBox upper left corner to be used in NMS and Box drawing
                x0 = int(Xc - (W / 2))
                y0 = int(Yc - (H / 2))
                
                BBox = [x0, y0, int(W), int(H)]
                BBoxs.append(BBox)
                confidences.append(float(confidence))
                classIDs.append(classID)
    
    ## Apply Non Maxima Suppression to remove duplicate overlapping boxes
    final_boxes_idxs = cv.dnn.NMSBoxes(BBoxs, confidences, 0.5, 0.4)
    
    # Load Coco dataset class names
    classes = open('coco.names').read().strip().split('\n')
    np.random.seed(1)
    colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')
    
    final_boxes = [BBoxs[i] for i in final_boxes_idxs.flatten()]
    final_classes= [classes[classIDs[i]] for i in final_boxes_idxs.flatten()]
    final_confidences=[confidences[i] for i in final_boxes_idxs.flatten()]

    # Create unique names for the detected objects based on the class names
    Objects=[]

    for i, obj in enumerate(final_classes):
        
        list=final_classes[:i+1]
        counter=list.count(obj)
        Objects=Objects+[obj+str(counter)]
        
    ## Draw the BBoxs and put the class labels on the image
    
    if len(final_boxes_idxs) > 0:
        for i in range(len(final_boxes)):
            (x, y) = (final_boxes[i][0], final_boxes[i][1])
            (w, h) = (final_boxes[i][2], final_boxes[i][3])
            color = [int(col) for col in colors[classIDs[i]]]
            cv.rectangle(img, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(Objects[i], final_confidences[i])
            cv.putText(img, text, (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    return img, final_boxes, Objects

## Normalized DLT

In [ ]:
def calc_new_points(W,H):

    dst = np.array([
        
            [0, 0],
            [W - 1, 0],
            [W - 1, H - 1],
            [0, H - 1]], dtype = "float32")
    
    return dst

def calc_new_points_given_p0(p0, W,H):
    
    x0=p0[0]
    y0=p0[1]

    dst = np.array([
        
            [x0, y0],
            [x0+W - 1, y0],
            [x0+W - 1, y0+H - 1],
            [x0, y0+H - 1]], dtype = "float32")
    
    return dst

def normalize(points):

    # Get the current centroid
    centroid=np.mean(points,0)

    # Get the average distance between the 4 points and the centroid
    distance=[]

    for point in points:
        distance+=[np.sqrt(((point[0] - centroid[0]) ** 2) + ((point[1] - centroid[1]) ** 2))]

    avg_distance=np.mean(distance)

    # Define the scaling factor
    s=np.sqrt(2)/avg_distance

    # Construct the similarity transformation matrix T

    trans_mat = np.matrix([[1, 0, -1*centroid[0]],[0, 1, -1*centroid[1]],[0, 0, 1]])
    scale_mat = np.matrix([[s, 0, 0],[0, s, 0],[0, 0, 1]])

    T=np.matmul(scale_mat, trans_mat)

    norm_points=[]

    # Calculate the new set of normalized points
    
    for point in points:
        x = np.array([point[0], point[1], 1]).reshape(3,1)

        x = np.matmul(T, x)

        norm_points += [[x.item(0), x.item(1)]]
        
    return (norm_points, T)

def dlt(points, new_points):
    
    A = []

    from scipy.linalg import svd

    # Construct the A matrix where each point pair forms 2 equations
    
    for p in range(len(points)):
        point=points[p]
        new_point=new_points[p]
        x, y = point[0],point[1]
        x_new, y_new = new_point[0],new_point[1]
        A.append( [-x, -y, -1, 0, 0, 0, x_new*x, x_new*y, x_new] )
        A.append( [0, 0, 0, -x, -y, -1, y_new*x, y_new*y, y_new] )

    # Convert A to array
    A = np.asarray(A) 

    # Calculate Singular-Value Decomposition of matrix A
    U, s, VT = svd(A)

    # Solution is last column of V

    V=VT.T
    H=V[:,-1]
    
    H=H.reshape(3, 3)  
    
    # Divide the solution by the last element h33
    
    H=H/H[-1, -1]
    
    return H

def dltnorm(points, newp0, W_new, H_new):
    
    # Get the new points
    new_points=calc_new_points_given_p0(newp0, W_new, H_new)
    
    # Get the normalized points and its transformation matrix
    normalized_points, T = normalize(points)
    
    # Get the normalized new points and its transformation matrix
    normalized_new_points, TT = normalize(new_points)
    
    # Apply the DLT to obtain the homography   
    H=dlt(normalized_points, normalized_new_points)
    
    # Denormalize the homography back    
    H = np.matmul( np.matmul(np.linalg.pinv(TT), H), T)
    
    H=H/H[-1, -1]
    
    return H

## Other Supportive Functions

In [ ]:
## Crop Image

def crop_image(img, box):
    
    (x, y) = (box[0], box[1])
    (w, h) = (box[2], box[3])
    
    cropped_image=img[y:y+h, x:x+w]

    p1=[x, y]
    p2=[x+w-1, y]
    p3=[x+w-1, y+h-1]
    p4=[x, y+h-1]   
    
    points=np.array([p1,p2,p3,p4])
    
    return cropped_image, points

## Check if image is colored or greyscale and return greyscale image

def image_preprocessing(img):
        im_pil = b64_to_pil(img)
        im_pil=np.array(im_pil)
            
        return (im_pil)
    
## Convert PIL image to b64

def pil_to_b64(im, enc_format='png', verbose=False, **kwargs):

    buff = _BytesIO()
    im.save(buff, format=enc_format, **kwargs)
    encoded = base64.b64encode(buff.getvalue()).decode("utf-8")

    return encoded

## Convert b64 string to PIL image

def b64_to_pil(string):
    decoded = base64.b64decode(string)
    buffer = _BytesIO(decoded)
    im = Image.open(buffer)

    return im

## Dash App.

In [ ]:
FA = "https://use.fontawesome.com/releases/v5.8.1/css/all.css"

## Set the path of the 'assets' directory
assets='C:/Users/toptech/Object Detection Toolbox/Dash_Sample/assets'

external_stylesheets = [assets+'/'+'bWLwgP.css']
external_stylesheets = [dbc.themes.BOOTSTRAP, FA]

app = dash.Dash(__name__,
                title='Object Detection Toolbox',
                assets_folder=assets,assets_url_path=assets,
                external_stylesheets=external_stylesheets,
                meta_tags = [{"name": "viewport", "content": "width=device-width, height=device-height, initial-scale=1.0"}],
                suppress_callback_exceptions=True)


SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 60,#53,
    "left": 0,
    "bottom": 0,
    "width": "170px",
    "padding": "1rem 1rem",
    "background-color": "#353535", #353535 #303030 #f8f9fa
}

CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

submenu_1 = [
    html.Li(
        dbc.Row(
            [
                dbc.Col('Main Menu',style={'color': '#FFFFFF','font-size': '16px', 'width':"auto"}),#Menu 1
                dbc.Col(
                    html.I(className="fas fa-chevron-right mr-3"), width="auto"
                ),
            ],
            className="my-1",
        ),
        id="submenu-1",
    ),

    dbc.Collapse(
        [
            dbc.Nav(
                    [
                        dbc.NavItem(dbc.NavLink("Object Detection", href="/page-1/1", style={'color': '#FFFFFF','font-size': '14px'})),
                        dbc.NavItem(dbc.NavLink("Cropping", href="/page-1/2", style={'color': '#FFFFFF','font-size': '14px'})),
                        dbc.NavItem(dbc.NavLink("Warping", href="/page-1/3", style={'color': '#FFFFFF','font-size': '14px'})),
                        
                                            ],vertical=True,pills=True,
            ),
        ],
        id="submenu-1-collapse",
        is_open=True
    )
]


sidebar = html.Div(
    [

        html.Hr(),
        dbc.Nav(submenu_1, vertical=True),
    ],
    style=SIDEBAR_STYLE,
    id="sidebar",
)


logo_button = dbc.Row(
    [

    ],

    className="ml-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

about_button = dbc.Button(
            "About", id="positioned-toast-about-toggle", color="danger", className="ml-auto"
        )

navbar = dbc.Navbar(
    [
        html.A(

            dbc.Row(
                [
                    dbc.Col(html.Img(src = app.get_asset_url('icon.png'),className = 'logo',height="50px",width="50px",style={"margin-left": "10px","margin-top": "5px","margin-right": "0px"}))
        ,
                    dbc.Col(dbc.NavbarBrand("Object Detection Toolbox", className="ml-2",style={'color': '#FFFFFF','font-size': '30px',"margin-left": "0px"})),
                ],
                align="left",

            ),
            href="/"
        ),
        html.A(

            dbc.Row(
                [
                    dbc.Col(html.H3(id='Page-Nav-Title',children=' ', className='ml-6',style={'color': '#00d4ff','font-size': '20px', 'margin-left':'100px'})),#,width={'size': 'Auto', 'order': 1, 'offset': 1}),
                ],
                align="center",

            ),

        ),
         dbc.NavbarToggler(id="navbar-toggler"),
        dbc.Collapse(dbc.Nav([about_button],className='ml-auto work-sans', navbar=True), id="navbar-collapse", navbar=True),

        logo_button
    ],
    color="#353535",
    dark=True,
)
modal = html.Div(
    [
        dbc.Button('Open modal', id='open'),
        dbc.Modal(
            [
                dbc.ModalHeader('Header'),
                dbc.ModalBody('This is the content of the modal'),
                dbc.ModalFooter(
                    dbc.Button('Close', id='close', className='ml-auto')
                ),
            ],
            id='modal',
        ),
    ]
)

about_toast = html.Div(
    [

        dbc.Toast(
            children='This is an Object Detection Toolbox',
            id='positioned-toast-about',
            header='About',
            is_open=False,
            dismissable=True,
            icon='danger',
            duration=5000,
            # top: 66 positions the toast below the navbar
            style={'position': 'fixed', 'top': 66, 'right': 20, 'width': 300},
        ),
    ]
)


content = html.Div(id="page-content", style=CONTENT_STYLE)

## Home Page

In [ ]:
############################# Page00 ####################################

UPLOAD_DIRECTORY = "/project/app_uploaded_files"

if not os.path.exists(UPLOAD_DIRECTORY):
       os.makedirs(UPLOAD_DIRECTORY)

def save_file(name, content):
    """Decode and store a file uploaded with Plotly Dash."""
    data = content.encode("utf8").split(b";base64,")[1]
    with open(os.path.join(UPLOAD_DIRECTORY, name), "wb") as fp:
        fp.write(base64.decodebytes(data))
        path=UPLOAD_DIRECTORY+'/'+name
        return path
         
@app.callback(
    Output("empty-image", "children"),
    [Input("upload-image", "filename"), Input("upload-image", "contents")],
)

def update_output(uploaded_filenames, uploaded_file_contents):
        return html.Div(id="empty-image")
    
@app.callback(
    Output("original-image", "data"),
    [Input("upload-image", "filename"), Input("upload-image", "contents")],
)


def update_output1(uploaded_filenames, uploaded_file_contents):
    """Save uploaded files and regenerate the file list."""

    if uploaded_filenames is not None and uploaded_file_contents is not None:
        for name, data in zip(uploaded_filenames, uploaded_file_contents):
            original_img_path=save_file(name, data)
        IMAGE_STRING_PLACEHOLDER = pil_to_b64(Image.open(original_img_path).copy(), enc_format='jpeg')
        return [name, IMAGE_STRING_PLACEHOLDER, uploaded_file_contents]


page0 = html.Div(
    children=[
    #navbar,
    html.Div([

        html.Div(id='logo_enter',children=[
        html.Img(src = app.get_asset_url('Logo1.png'),height = '200 px',width = '1000')
        ],style={'margin-left':'75px','float': 'center','align': 'center', 'display': 'inline-block', 'max-height' : '200 px', 'height' : 'auto', 'width' : 'auto','textAlign': 'center'}
        ),
         html.Div([
            dcc.Upload(
                id='upload-image',
                children=html.Div([
                    'Drag and Drop or ',
                    html.A('Select Image')
                ]),
                style={'align': 'center',
                    'width': '90%',
                    'height': '100px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px', 'margin-top':'50px','font-size': '20px', "font-weight": "bold", 'margin-left':'60px'
                },
                # Allow multiple files to be uploaded
                multiple=True
        )]),
     html.Div(
    [   dcc.Link(dbc.Button(
            "Start", id="start-button", className="d-grid gap-2 col-2 mx-auto", color="secondary",
            style={'text-align': 'center','font-size': '20px', "font-weight": "bold",'margin-top':'30px'}), href='/page-0/0'),
     html.Div(id="empty-image"),
    ]
    )
    ])
])


#########################################################################


## Page00 - Display Original Image

In [ ]:
page00 = html.Div(children=html.Div(id='original-image-fig',style={"width": "80%", "display": "inline-block", "padding": "0 0"}))

@app.callback(Output('original-image-fig', 'children'),
              Input('original-image', 'data'))

def display_img(img):

    im_content=img[2]

    config = {
        "modeBarButtonsToAdd": [
            "drawline",
            "drawopenpath",
            "drawclosedpath",
            "drawcircle",
            "drawrect",
            "eraseshape",
        ]
    }

    return html.Div(children=[html.H1("Original Image", style={'textAlign': 'left', 'margin-top':'20px', 'margin-bottom':'20px'}),
                               
                               html.Img(src=im_content, style={'allign': 'left', 'height':"auto",'width':"400px"}),
                               ])


## Page1 - Object Detection

In [ ]:
page1 = html.Div(children=[
        dbc.Row(
            id="header1",
            children=[
            html.H4(children='Please wait: YOLO is processing the image ...',style={'display': 'inline-block','margin-top':'20px','margin-left':'20px', 'textAlign': 'center', 'font-size': '24px', 'horizontalAlign':'center'}),
        ],style={'display': 'inline-block', 'horizontalAlign':'center', 'textAlign': 'center'}
        ),

      html.Div(children=html.Div(id='object-graph'))

])

@app.callback(Output('object-graph', 'children'),
              Output('header1', 'style'),
              Output('output-image', 'data'),
              Output('boxes-store', 'data'),
              Output('objects-store', 'data'),
              Input('original-image', 'data'))

def update_graph(img):

        
    im_pil = image_preprocessing(img[1])
    original_fig=px.imshow(im_pil, binary_string=False)
    original_fig.update_layout(title_text='<b>Original Image<b>', title_x=0.5)

    output_img, boxes, objects = YOLO(im_pil)
    output_fig=px.imshow(output_img, binary_string=False)
    output_fig.update_layout(title_text='<b>YOLO Object Detector Output<b>', title_x=0.5)
            
    object_graph = html.Div(children=[
            html.Div(children=[dcc.Graph(id="image_orig", figure=original_fig)], style={"width": "50%", "display": "inline-block", "padding": "0 0", "verticalAlign": "top","horisontalAlign": "right", "hight": "auto"}),
            html.Div(children=[dcc.Graph(id="output", figure=output_fig),],style={"width": "50%", "display": "inline-block", "padding": "0 0", "horisontalAlign": "left", "hight": "auto"})])

    return [object_graph, {'display': 'none'}, [output_img], boxes, objects]

## Page 2 - Cropping

In [ ]:
page2 = html.Div(style={'backgroundColor':'#f4f4f2'},
    children=[
    #navbar,
    html.Div([
        html.Div(
            id="header2",
            children=[
            html.H4(children='Please select the object of interest',style={'margin-left':'10px', 'margin-top':'5px'}),
        ],style={'display': 'inline-block', 'margin-bottom':'5px'}
        ),

    ]),
        

        dbc.Row(children=[
            dcc.Dropdown(
                id='object-filter1', style={'width': '300px', 'display': 'inline-block','margin-left':'10px','margin-right':'10px','font-size': '14px'}),
        
     
        dbc.Button(
            "Crop", id="crop-button1", className="d-grid col-2 mx-auto", color="secondary",
            style={'text-align': 'center','font-size': '20px', "font-weight": "bold", 'margin-right':'10px', 'display': 'inline-block',"horizontalAlign": "left",'margin-top':'0px'}),

        ]),
        
        html.Div(children=html.Div(id='output-graph1'),style={'margin-top':'10px'})
 

        ])

@app.callback(
    dash.dependencies.Output('object-filter1', 'options'),
    [dash.dependencies.Input('objects-store', 'data')]
)
def update_object_dropdown1(objects):
    options=[{'label':val, 'value':opt} for opt,val in enumerate(objects)]
    return options



@app.callback(
    [Output('output-graph1', 'children'),
     Output('crop-button1', 'n_clicks'),
     Input('object-filter1', 'value'),
     Input('boxes-store', 'data'),
     Input('original-image', 'data'),
     Input('output-image', 'data'),
     Input('crop-button1', 'n_clicks'),
    ])
        
def update_crop_graph1(object_id,boxes, img, output_img, crop_n_click):
    
    output_img = np.array(output_img[0], dtype=np.uint8)

    output_fig=px.imshow(output_img, binary_string=False)
    output_fig.update_layout(title_text='<b>YOLO Object Detector Output<b>', title_x=0.5)

    
    if ((crop_n_click is None) | (crop_n_click==0)):
        
        output_graph = html.Div(children=[
            html.Div(children=[dcc.Graph(id="image_out1", figure=output_fig)], style={"width": "50%", "display": "inline-block", "horisontalAlign": "left", "hight": "auto"})
        ])

    elif crop_n_click>=1:
        
        im_pil = image_preprocessing(img[1])
        box=boxes[object_id]
        cropped_img, points=crop_image(im_pil, box)
        crop_fig=px.imshow(cropped_img, aspect='auto')
        crop_fig.update_layout(title_text='<b>Cropped Object<b>', title_x=0.5)

        output_graph = html.Div(children=[
            html.Div(children=[dcc.Graph(id="image_out3", figure=output_fig)], style={"width": "50%", "display": "inline-block", "hight": "auto"}),
            html.Div(children=[dcc.Graph(id="image_crop1", figure=crop_fig)],style={"width": "50%", "display": "inline-block",  "hight": "auto"})])
  
    return [output_graph, 0]

## Page3 - Warping

In [ ]:
page3 = html.Div(style={'backgroundColor':'#f4f4f2'},
    children=[
    #navbar,
    html.Div([
        html.Div(
            id="header2",
            children=[
            html.H4(children='Please select the object of interest',style={'margin-left':'10px', 'margin-top':'5px'}),
        ],style={'display': 'inline-block', 'margin-bottom':'5px'}
        ),

    ]),
        

        dbc.Row(children=[
            dcc.Dropdown(
                id='object-filter', style={'width': '300px', 'display': 'inline-block','margin-left':'10px','margin-right':'10px','font-size': '14px'}),
        
     
        dbc.Button(
            "Select", id="select-button", className="d-grid col-2 mx-auto", color="secondary",
            style={'text-align': 'center','font-size': '20px', "font-weight": "bold", 'margin-right':'10px', 'display': 'inline-block',"horizontalAlign": "left",'margin-top':'0px'}),

        ]),
             
            dbc.Row(html.Div(id='warp-para', children=
            [html.H1("Width : ", style={'font-size': '12px',"font-weight": "bold",'textAlign': 'left', 'margin-left':'20px','margin-top':'10px','display': 'inline-block'}),
            html.Div(dcc.Input(id='w',placeholder="input type float", type='number',value=0.0, style={'margin-left':'10px', 'width':'auto' ,'display': 'inline-block'})
                    ,style={'display': 'inline-block','margin-left':'10px', 'margin-top':'5px'}),

            html.H1("Height : ", style={'font-size': '12px',"font-weight": "bold",'textAlign': 'left', 'margin-left':'20px','margin-top':'10px','display': 'inline-block'}),
            html.Div(dcc.Input(id='h',placeholder="input type float", type='number',value=0.0, style={'margin-left':'10px', 'width':'auto' ,'display': 'inline-block'})
                    ,style={'display': 'inline-block'}),
            
            html.Div(dbc.Button(
            "Warp", id="warp-button", className="d-grid col-6 mx-auto", color="secondary",
            style={'text-align': 'center','font-size': '20px', "font-weight": "bold"}),style={'text-align': 'center','font-size': '20px', "font-weight": "bold", 'margin-left':'110px', 'display': 'inline-block',"horizontalAlign": "right",'margin-top':'0px','width': '390px'})
             
                    ],style={'display': 'none', 'margin-left':'20px', 'margin-top':'5px'}))
        
        ,html.Div(id='display-selected-values', children=[
            html.H6(children='Please select the object of interest',style={'margin-left':'10px', 'margin-top':'5px', 'font-size': '12px'}),
        ],style={'display': 'none', 'margin-top':'10px', 'margin-left':'10px', 'font-size': '12px'}
        )
        
        ,html.Div(children=html.Div(id='output-graph'),style={'margin-top':'10px'})
 

        ])


@app.callback(
    Output("where", "children"),
    Output("points", "data"),
    Input("image_crop", "clickData"),
    Input("points", "data"),
    Input('select-button', 'n_clicks'))

def click(clickData, lpoints, select_n_clicks):
    
    if select_n_clicks>0:
        return (None,None)
    elif not clickData:
        raise dash.exceptions.PreventUpdate

    elif lpoints is None:
        lpoints=[]
        lpoints=lpoints+[[clickData["points"][0]["y"],clickData["points"][0]["x"]]]

        return (lpoints,lpoints)
    else:
        lpoints=lpoints+[[clickData["points"][0]["y"],clickData["points"][0]["x"]]]

        return (lpoints,lpoints)
    
@app.callback(
    Output('display-selected-values', 'children'),
    Input("points", "data"))

def set_display_children(points):
    
    return u'The points that you have selected are: {}'.format(points)

@app.callback(
    dash.dependencies.Output('object-filter', 'options'),
    [dash.dependencies.Input('objects-store', 'data')]
)
def update_object_dropdown(objects):
    options=[{'label':val, 'value':opt} for opt,val in enumerate(objects)]
    return options



@app.callback(
    [Output('output-graph', 'children'),
     Output('warp-para', 'style'),
     Output('display-selected-values', 'style'),
     Output('w', 'value'),
     Output('h', 'value'),
     Output('select-button', 'n_clicks'),
     Output('warp-button', 'n_clicks'),
     Input('object-filter', 'value'),
     Input('boxes-store', 'data'),
     Input('original-image', 'data'),
     Input('select-button', 'n_clicks'),
     Input('warp-button', 'n_clicks'),
     Input('w', 'value'),
     Input('h', 'value'),
     Input("points", "data")])
        
def update_crop_graph(object_id,boxes, img, select_n_click,warp_n_click, w, h, warppoints):
    
    if ((select_n_click is None) | (((warp_n_click is None) | (warp_n_click==0)) and select_n_click==0)):
        
        raise dash.exceptions.PreventUpdate

    elif (((warp_n_click is None) | (warp_n_click==0)) and select_n_click>0):
        
        w=0.0
        h=0.0
        
        im_pil = image_preprocessing(img[1])
        
        box=boxes[object_id]
        cropped_img, points=crop_image(im_pil, box)
        crop_fig=px.imshow(cropped_img,binary_string=True, aspect='auto')
        x_dim=cropped_img.shape[0]
        y_dim=cropped_img.shape[1]
        crop_fig.update_layout(title_text='<b>Selected Object<b>', title_x=0.5)
        crop_fig.add_traces(
                px.scatter(
                    x=np.repeat(np.linspace(0, x_dim-1, x_dim), y_dim), y=np.tile(np.linspace(0 ,y_dim-1, y_dim), x_dim), opacity=0
                ).update_traces(marker_color="rgba(0,0,0,0)", marker_size=0).update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)').data)

        output_graph = html.Div(children=[
                         html.Div(children=[dcc.Graph(id="image_crop", figure=crop_fig)], style={"width": "50%", "display": "inline-block", "padding": "0 0", "verticalAlign": "top","horisontalAlign": "right", "hight": "auto"}),
                         html.Div(id="where",style={ "display": 'inline'})])

    elif (select_n_click>=0 and (warp_n_click>=1)):
        
               
        im_pil = image_preprocessing(img[1])
        box=boxes[object_id]

        cropped_img, points=crop_image(im_pil, box)
        x_dim=cropped_img.shape[0]
        y_dim=cropped_img.shape[1]
        crop_fig=px.imshow(cropped_img,binary_string=True, aspect='auto')
        crop_fig.update_layout(title_text='<b>Selected Object<b>', title_x=0.5)
        crop_fig.add_traces(
                px.scatter(
                    x=np.repeat(np.linspace(0, x_dim-1, x_dim), y_dim), y=np.tile(np.linspace(0 ,y_dim-1, y_dim), x_dim), opacity=0
                ).update_traces(marker_color="rgba(0,0,0,0)", marker_size=0).update_layout(paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)').data)

        warppoints=[[warppoints[0][1],warppoints[0][0]],
                   [warppoints[1][1],warppoints[1][0]],
                    [warppoints[2][1],warppoints[2][0]],
                    [warppoints[3][1],warppoints[3][0]]]
        
        tl=warppoints[0]
        tr=warppoints[1]
        br=warppoints[2]
        bl=warppoints[3]

        widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
        widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
        maxWidth = max(int(widthA), int(widthB))  

        heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
        heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
        maxHeight = max(int(heightA), int(heightB))
        
        if ((w==0.0) and (h==0.0)):
            (w, h) = (maxWidth, maxHeight)

        homography = dltnorm(warppoints, (0,0), w, h)
        warped_img_cropped = cv.warpPerspective(cropped_img,homography,(w, h))
 
        warped_img_cropped_fig=px.imshow(warped_img_cropped,binary_string=True, aspect='auto')
        
        homography = dltnorm(warppoints, tl, w, h)
        warped_img_full = cv.warpPerspective(cropped_img,homography,(y_dim, x_dim))
 
        warped_img_full_fig=px.imshow(warped_img_full,binary_string=True, aspect='auto')
        warped_img_full_fig.update_layout(title_text='<b>Warped Image<b>', title_x=0.5)
        
        

        output_graph = html.Div(children=[
                    html.Div(children=[dcc.Graph(id="image_crop", figure=crop_fig)], style={"width": "50%", "display": "inline-block", "padding": "0 0", "verticalAlign": "top","horisontalAlign": "right", "hight": "auto"}),
                    html.Div(children=[dcc.Graph(id="warped_img_full", figure=warped_img_full_fig)],style={"width": "50%", "display": "inline-block", "padding": "0 0", "horisontalAlign": "left", "hight": "auto"})
                    ,html.Div(children=[dcc.Graph(id="warped_img_cropped", figure=warped_img_cropped_fig)],style={"width": "50%", "display": "inline-block", "padding": "0 0", "horisontalAlign": "right", "hight": "auto",'margin-left':'300px'})
                    ,html.Div(id="where",style={ "display": 'inline'})])

    return [output_graph,{'display': 'inline-block'},{'display': 'inline-block', 'margin-top':'10px', 'margin-left':'10px', 'font-size': '12px'}, w, h, 0, 0]


## App Layout

In [ ]:
app.layout = html.Div([dcc.Location(id='url'), sidebar,navbar,about_toast, content, dcc.Store(id='original-image'), dcc.Store(id='boxes-store'), dcc.Store(id='objects-store'), dcc.Store(id='output-image'), dcc.Store(id='points')])


# this function is used to toggle the is_open property of each Collapse
def toggle_collapse(n, is_open):
    if n:
        return not is_open
    return is_open


# this function applies the "open" class to rotate the chevron
def set_navitem_class(is_open):
    if is_open:
        return 'open'
    return ''


for i in [1, 2]:
    app.callback(
        Output(f'submenu-{i}-collapse', 'is_open'),
        [Input(f'submenu-{i}', 'n_clicks')],
        [State(f'submenu-{i}-collapse', 'is_open')],
    )(toggle_collapse)

    app.callback(
        Output(f'submenu-{i}', 'className'),
        [Input(f'submenu-{i}-collapse', 'is_open')],
    )(set_navitem_class)

@app.callback(
    Output('positioned-toast-about', 'is_open'),
    [Input('positioned-toast-about-toggle', 'n_clicks')],
)
def open_toast(n):
    if n:
        return True
    return False


@app.callback([Output('page-content', 'children'), Output('Page-Nav-Title','children')], [Input('url', 'pathname')])
def render_page_content(pathname):
    if pathname in ['/']:
        return page0,'' 
    elif pathname == '/page-0/0':
        return page00,''
    elif pathname == '/page-1/1':
        return page1,'Object Detection'
    elif pathname == '/page-1/2':
        return page2,'Cropping'
    elif pathname == '/page-1/3':
        return page3,'Warping'
    
    # If the user tries to reach a different page, return a 404 message
    return dbc.Jumbotron(
        [
            html.H1('404: Not found', className='text-danger'),
            html.Hr(),
            html.P(f'The pathname {pathname} was not recognised...'),
        ]
    )


# add callback for toggling the collapse on small screens
@app.callback(
    Output('navbar-collapse', 'is_open'),
    [Input('navbar-toggler', 'n_clicks')],
    [State('navbar-collapse', 'is_open')],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open



if __name__ == '__main__':
    app.run_server(debug=False)
